In [1]:
import numpy as np
from numpy.random import default_rng
import cv2
from scipy.signal import convolve
from scipy.special import expit

rng = default_rng()

import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
# from plotoptix import TkOptiX
# # from plotoptix import NpOptiX

# from plotoptix.materials import m_clear_glass, m_mirror, m_plastic
# from plotoptix.utils import map_to_colors

In [15]:
# text_grid = np.zeros((text_image.shape[0], text_image.shape[1], 16), dtype=int)
# for idx in range(7, 8):
#     text_grid[:, :, idx] += text_image

# text_grid.sum()

961

In [16]:
# optix = TkOptiX()

# optix.setup_material("glass", m_clear_glass)
# optix.setup_material("plastic", m_plastic)
# m_clear_glass["VarFloat"]["radiation_length"] = 0.95
# m_clear_glass["VarFloat3"]["subsurface_color"] = [ 0.7, 0.85, 1 ]
# m_clear_glass["VarFloat3"]["surface_albedo"] = [ 0.6, 0.8, 1 ]     # add some color to reflections
# optix.update_material("glass", m_clear_glass)


# optix.set_param(min_accumulation_step=16,
#                 max_accumulation_frames=256)

# # data = np.argwhere(text_grid)
# data = np.argwhere(gol_sim.grid)

# cc = map_to_colors(data[:, 1], "brg")

# optix.set_data(
#     "my plot",
#     pos=data,
#     u=np.array([0.95, 0, 0]),
#     v=np.array([0, 0.95, 0]),
#     w=np.array([0, 0, 0.95]),
#     geom="Parallelepipeds",
#     c=cc,
#     mat="plastic",
# )

# optix.set_data("plane", geom="Parallelograms", c=0.99,
#                 pos=[0, 0, 0], u=[100, 0, 0], v=[0, 200, 0], w=[0, 0, 500])

# optix.set_ambient([0.4, 0.6, 0.8])

# exposure = 1.0
# gamma = 1.0
# optix.set_float("tonemap_exposure", exposure)
# optix.set_float("tonemap_gamma", gamma)
# optix.set_float("denoiser_blend", 0.25)
# optix.add_postproc("Denoiser")
# optix.add_postproc("Gamma")

# # Show coordinate box.
# # optix.set_coordinates()

# optix.set_background(0.99)
# optix.set_ambient(0.85)

# optix.setup_camera(
#     "cam1",
#     cam_type="ThinLens",
#     eye=np.array([3, 0.25, 2.2]) * 70,
#     target=data.mean(axis=0).astype(int),
#     up=[0, 0, 1],
#     fov=18,
#     focal_scale=.95)

# optix.start()

	Selected devices: [0]
	Primary device: 0


In [2]:
# conv_array = np.array([
#     [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
#     [[0, 1, 0], [1, 1, 1], [0, 1, 0]],
#     [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
# ])

# plt.figure(figsize=(5,5)).add_subplot(projection='3d').voxels(conv_array);

In [3]:
class GoLSimulation:

    def __init__(self, grid):
        self.grid = grid.astype(int)
        self.grid_history = list([grid])

    def step(self, x):
        conv_array = np.ones((3, 3, 3))
        nbrs_count = convolve(x, conv_array, mode='same') - x
        return ((nbrs_count >= 3) | (x & (nbrs_count >= 2))) & (nbrs_count <= 5)

    def simulate(self, n_iter:int):
        for _ in range(n_iter):
            new_grid = self.step(self.grid)
            self.grid_history.append(new_grid)
            self.grid = new_grid
        return self

    def _repr_png_(self):
        plt.figure(figsize=(5,5)).add_subplot(projection='3d').voxels(self.grid)


# grid_3d = np.zeros((25, 25, 25))
# random_pop = (rng.random(grid_3d.shape) > 0.95).astype(int)

# gol_sim = GoLSimulation(grid=random_pop)
# gol_sim = gol_sim.simulate(5)
# gol_sim

In [4]:
image = np.zeros((100, 150, 3), np.uint8)

position = (25, 60)

cv2.putText(image, "Praxis",
            position,
            cv2.FONT_HERSHEY_PLAIN, #font family
            2, #font size
            (255, 255, 255, 255), #font color
            3)
text_image = (image[:, :, 0] > 0).astype(int)
text_grid = np.zeros((text_image.shape[0], text_image.shape[1], 16), dtype=int)
for idx in range(7, 8):
    text_grid[:, :, idx] += text_image

text_grid.sum()

961

In [5]:
%%time
gol_sim = GoLSimulation(grid=text_grid)
gol_sim = gol_sim.simulate(12)

CPU times: user 302 ms, sys: 68 ms, total: 370 ms
Wall time: 369 ms


In [6]:
# gol_sim

In [7]:
# from plotoptix import TkOptiX
from plotoptix import NpOptiX
from plotoptix.materials import m_clear_glass, m_mirror, m_plastic
from plotoptix.utils import map_to_colors

	[0]: NVIDIA GeForce GTX 970


In [8]:
def size_flux(gen, n_frames, sim=gol_sim):
    if gen == 0:
        prev_pos = np.zeros_like(sim.grid_history[0])
    else:
        prev_pos = sim.grid_history[gen - 1]
    curr_pos = sim.grid_history[gen]
    
    stable_cells = np.argwhere((prev_pos == 1) & (curr_pos == 1))
    new_cells    = np.argwhere((prev_pos == 0) & (curr_pos == 1))
    dying_cells  = np.argwhere((prev_pos == 1) & (curr_pos == 0))
    
    # starts = rng.choice(np.arange(4, 6), new_cells.shape[0])
    # stops = rng.choice(np.arange(n_frames - 4, n_frames), new_cells.shape[0])
    stable_cell_sizes = (1 - np.abs(np.sin(np.linspace(0, np.pi, n_frames)[:, np.newaxis])) * 0.15 * np.ones(stable_cells.shape[0]))
    # stable_cell_sizes += rng.normal(size=stable_cell_sizes.shape[0])[:, np.newaxis] * 0.02
    
    starts = rng.choice(np.arange(4, 6), new_cells.shape[0])
    stops = rng.choice(np.arange(n_frames - 4, n_frames), new_cells.shape[0])
    xx = np.linspace(starts, stops, n_frames) - (n_frames/2)
    new_cell_sizes = expit(xx)


    starts = rng.choice(np.arange(4, 6), dying_cells.shape[0])
    stops = rng.choice(np.arange(n_frames - 4, n_frames), dying_cells.shape[0])
    xx = np.linspace(stops, starts, n_frames) - (n_frames/2)
    dying_cell_sizes = expit(xx)

    pos = np.vstack((stable_cells, new_cells, dying_cells))
    sizes = np.hstack((stable_cell_sizes, new_cell_sizes, dying_cell_sizes))
    # sizes += rng.normal(size=sizes.shape[0])[:, np.newaxis] * 0.05
    return pos, sizes


class params:
    sim = gol_sim
    n_gen = 12
    gen_time = 2
    fps = 24
    width = 1920
    height = 1080
    
    max_scale = 0.95
    u = np.array([1, 0, 0]) * 1 /24
    v = np.array([0, 1, 0]) * 1 /24
    w = np.array([0, 0, 1]) * 1 /24
    
    pos, gen_uvw = size_flux(0, gen_time * fps)
    
    colors = map_to_colors(pos[:, 1], "brg")
    frame = 0
    
    # Add some fancy camera jitter.
    eye = np.array([3, 0.25, 2.2]) * 70
    camera_x = eye[0]
    camera_z = eye[2]
    _tt = np.linspace(0, 2*np.pi, num=gen_time * fps * n_gen)
    # print(_tt.shape)
    camera_xx = 3 * np.cos(_tt) + camera_x
    camera_zz = 8 * np.sin(_tt) + camera_z
    
    ffov = np.linspace(16, 20, gen_time * fps * n_gen)
    fov = 16


def init(rt):
    rt.set_param(min_accumulation_step=16, max_accumulation_frames=500)
    rt.set_float("tonemap_exposure", 1.15)
    rt.set_float("tonemap_gamma", 1.0)
    rt.set_float("denoiser_blend", 0.15)
    rt.add_postproc("Denoiser")
    rt.add_postproc("Gamma")
    rt.set_background(0.99)
    rt.set_ambient(0.85)
    rt.setup_material("plastic", m_plastic)
    rt.set_data("plot", pos=params.pos, u=params.u, v=params.v, w=params.w,
                geom="Parallelepipeds", c=params.colors, mat="plastic")
    # Add a plane.
    rt.set_data("plane", geom="Parallelograms", c=0.99,
                pos=[-1000, 0, -1], u=[5000, 0, 0], v=[0, 1000, 0], w=[0, 0, 1000])
    rt.setup_camera("cam1", cam_type="ThinLens", eye=params.eye,
                    target=[50, 75, 6], up=[0, 0, 1], fov=18, focal_scale=.95)
    


    
    
def compute(rt: NpOptiX, delta: int) -> None:
    params.frame += 1
    gen = params.frame // (params.gen_time * params.fps)
    gen_frame = params.frame % (params.gen_time * params.fps)
    
    if params.frame % (params.gen_time * params.fps) == 0:
        # Update generation sizes.
        pos, gen_uvw = size_flux(gen, params.gen_time * params.fps)
        params.pos = pos
        params.gen_uvw = gen_uvw
        
    # params.pos = params.pos + params.gen_uvw[gen_frame, :][:, np.newaxis] * params.max_scale
    scale_broadcast = params.gen_uvw[gen_frame, :][:, np.newaxis] * params.max_scale
    
    params.camera_x = params.camera_xx[params.frame]
    params.camera_z = params.camera_zz[params.frame]
    
    params.u = np.array([1, 0, 0]) * scale_broadcast
    params.v = np.array([0, 1, 0]) * scale_broadcast
    params.w = np.array([0, 0, 1]) * scale_broadcast
    params.colors = map_to_colors(params.pos[:, 1], "brg")
    params.fov = params.ffov[params.frame]
    

    



def update(rt: NpOptiX) -> None:
    rt.update_data("plot", pos=params.pos, u=params.u, v=params.v, w=params.w, c=params.colors)
    rt.update_camera("cam1", eye=[params.camera_x , 35, params.camera_z], fov=params.fov)
    if params.frame % params.fps == 0:
        print(f'frame: {params.frame}')
        rt.save_image("output/frame_{:05d}.png".format(params.frame))

    
    


optix = NpOptiX(on_initialization=init,
                on_scene_compute=compute,
                on_rt_completed=update,
                width=params.width, 
                height=params.height,
                start_now=False)


optix.encoder_create(fps=params.fps, profile="High")
optix.start()
optix.encoder_start("output/gol_HD2.mp4", params.gen_time * params.fps * params.n_gen)

	Selected devices: [0]
	Primary device: 0
[INFO ][11:24:29] Encoding Parameters:
	codec        : h264
	preset       : default
	profile      : high(h264)
	chroma       : yuv420
	bitdepth     : 8
	rc           : constqp (P,B,I=28,31,25)
	fps          : 24/1
	gop          : 48
	bf           : 0
	size         : 1920x1080
	bitrate      : 0
	maxbitrate   : 0
	vbvbufsize   : 0
	vbvinit      : 0
	aq           : disabled
	temporalaq   : disabled
	lookahead    : disabled
	cq           :  
	qmin         : P,B,I=0,0,0
	qmax         : P,B,I=0,0,0
	initqp       : P,B,I=0,0,0


In [9]:
print(optix.encoding_frames(), optix.encoded_frames())

576 0
frame: 24
frame: 48
frame: 72
frame: 96
frame: 120
frame: 144
frame: 168
frame: 192
frame: 216
frame: 240
frame: 264
frame: 288
frame: 312
frame: 336
frame: 360
frame: 384
frame: 408
frame: 432
frame: 456
frame: 480
frame: 504
frame: 528
frame: 552


Exception ignored on calling ctypes callback function: <function NpOptiX._get_start_scene_compute_callback.<locals>.func at 0x7f0212ec00d0>
Traceback (most recent call last):
  File "/home/tyer/anaconda3/envs/pymc/lib/python3.8/site-packages/plotoptix/npoptix.py", line 796, in func
    def func(n_frames : int): self._start_scene_compute_callback(n_frames)
  File "/home/tyer/anaconda3/envs/pymc/lib/python3.8/site-packages/plotoptix/npoptix.py", line 794, in _start_scene_compute_callback
    for c in self._scene_compute_cb: c(self, n_frames)
  File "/tmp/ipykernel_35221/663028693.py", line 101, in compute
IndexError: index 576 is out of bounds for axis 0 with size 576
Exception ignored on calling ctypes callback function: <function NpOptiX._get_scene_rt_completed_callback.<locals>.func at 0x7f0212ec0160>
Traceback (most recent call last):
  File "/home/tyer/anaconda3/envs/pymc/lib/python3.8/site-packages/plotoptix/npoptix.py", line 839, in func
    def func(rt_result : int): self._scene_

In [12]:
optix.encoder_stop()

In [13]:
optix.close()

AssertionError: Raytracing output already closed.

In [13]:
# %%bash

# ffmpeg -i 'output/gol_6.mp4' -vf reverse 'output/gol_6r.mp4'